In [41]:
import sys
import pandas as pd
from sklearn.metrics import roc_auc_score
#Own library
import library_hw4
from library_hw4.lib4 import load_class as ld
from library_hw4.lib4 import prepro_class as pr
from library_hw4.lib4 import feature_class as ft
from library_hw4.lib4 import model_class as mc

#### Load Train and Test data

In [2]:
loader = ld.diabetes_loader_jupyter()

In [3]:
train, test = loader.train_and_test_data()

#### Preprocessing

In [4]:
# Create missing remover objects for train and test
missing_remover = pr.DropMissingValuesGenderEthnicity(train)
missing_remover_t = pr.DropMissingValuesGenderEthnicity(test)

In [5]:
# Remove missing rows from columns Age, gender, Ethnicity
train_clean = missing_remover.process()
test_clean = missing_remover_t.process()

In [6]:
# Create Imputation of missing values object for train and test
missing_imputator = pr.FillMissingValuesWithMeanHeightWeight(train_clean)
missing_imputator_t = pr.FillMissingValuesWithMeanHeightWeight(test_clean)

In [7]:
# Impute missing values for height and weight
train_imp = missing_imputator.process()
test_imp = missing_imputator_t.process()

#### Feature transformation

In [8]:
# One hot enco object
feature_trans = ft.One_hot_enc(train_imp)
feature_trans_t = ft.One_hot_enc(test_imp)

In [9]:
# Create new dummy columns based on icu_admit_source
train_new_f = feature_trans.one_hot_enc('icu_admit_source')
test_new_f = feature_trans_t.one_hot_enc('icu_admit_source')

object
object


In [10]:
# Create object for replacing boolean to 1-0
boolean_trans = ft.replace_booleans(train_new_f)
boolean_trans_t = ft.replace_booleans(test_new_f)

In [11]:
# Transform gender variable to binary
final_train = boolean_trans.input_mean_weight('gender')
final_test = boolean_trans_t.input_mean_weight('gender')

#### Model train and prediction

In [12]:
# Feature columns
X = ["age", "height", "weight", "aids", "cirrhosis", "hepatic_failure", 
     "immunosuppression", "leukemia", "lymphoma", "solid_tumor_with_metastasis","gender",
     'icu_admit_source_Accident & Emergency', 'icu_admit_source_Floor', 'icu_admit_source_Operating Room / Recovery', 'icu_admit_source_Other Hospital', 'icu_admit_source_Other ICU']
# Target column
Y = "diabetes_mellitus"

In [13]:
# Create logistic model class
logistic_model = mc.CustomModel(final_train,X,Y,'Logistic')

In [14]:
#Train model
logistic_model.train()

In [16]:
# Predict test values
probabilities, predictions = logistic_model.prediction(final_test[X])

In [37]:
# Add a new column "predictions" to final_test dataset
final_test['probabilities'] = probabilities.tolist() # Assuming you're interested in the probabilities of the positive class
final_test['predictions'] = predictions.tolist()  # Final prediction [0,1]

In [42]:
# Compute the ROC AUC score (substitute )
roc_auc = roc_auc_score(final_test['diabetes_mellitus'], final_test['predictions'])
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.5186346482571823
